In [1]:
import numpy.random as rd

PERSON_NUM = 30
VALUE_NUM = 12

person = []
for i in range(PERSON_NUM):
    value = rd.randint(low=0, high=2, size=VALUE_NUM)
    person.append(value)

In [2]:
import numpy as np

def calc_weight(a, b):
    distance = np.linalg.norm(a-b)
    weight = 1/(1+distance)
    return weight

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import networkx as nx
import itertools

G = nx.Graph()

nodes_list = [i for i in range(PERSON_NUM)]
G.add_nodes_from(nodes_list)

for edge in list(itertools.combinations(nodes_list, 2)):
    G.add_path(edge, weight=calc_weight(person[edge[0]], person[edge[1]]))
    
# pos=nx.spring_layout(G)

# nx.draw_networkx_nodes(G, pos)
# nx.draw_networkx_labels(G, pos)

# edge_width = [ d["weight"] for (u,v,d) in G.edges(data=True)]
# nx.draw_networkx_edges(G, pos, width=edge_width)

# plt.show()

In [4]:
d = nx.max_weight_matching(G)

In [5]:
# 選ばれたエッジ
matching_group = d
# 上限に達したノード
selected_nodes = set()

In [6]:
def reMatching(matching_group, d, selected_nodes):
    
    # 選ばれたエッジを削除
    G.remove_edges_from(d)
    
    # 上限に達したノードの更新
    for group in matching_group:
        if len(group) == 4:
            selected_nodes = selected_nodes | set(group)
    
    # 上限に達したノードを削除
    G.remove_nodes_from(selected_nodes)

    # 再度最大流マッチングを実行
    d = nx.max_weight_matching(G)

    for dd in d:
        i, j = dd
        i_group = tuple({i})
        j_group = tuple({j})

        for group in matching_group:
            if i in group:
                i_group = group
            if j in group:
                j_group = group

        new_group = tuple(set(i_group) | set(j_group))
        
        if len(new_group) <= 4:

            matching_group.add(new_group)
            matching_group.discard(i_group)
            matching_group.discard(j_group)
    
    return matching_group, d, selected_nodes

In [7]:
import time
start = time.time()

In [8]:
while (len(d) > 0):
    matching_group, d, selected_nodes = reMatching(matching_group, d, selected_nodes)

In [9]:
# 半端な人数に応じた処理
rest_node = set(nodes_list) - selected_nodes

if len(rest_node) == 1:
    # 選ばれなかった一人をどこかに組み込む
    for group in matching_group:
        new_group = group + tuple(rest_node)
        matching_group.add(new_group)
        matching_group.discard(group)
        break
    
elif len(rest_node) == 2:
    # 2人組を分解して、それぞれどこかに組み込む
    for group in matching_group:
        if len(group) == 2:
            i, j = group
            matching_group.discard(group)
            break
    
    for group in matching_group:
        new_group = group + tuple({i})
        matching_group.add(new_group)
        break

    for group in matching_group:
        if len(group) == 4:
            new_group = group + tuple({j})
            matching_group.add(new_group)
            break
    
elif len(rest_node) == 5:
    # 2人組と3人組をペアにする
    for group in matching_group:
        if len(group) == 2:
            two_group = group
        if (group) == 3:
            three_group = group
    
    new_group = two_group + three_group
    matching_group.add(new_group)
    matching_group.discard(two_group)
    matching_group.discard(three_group) 

print(matching_group)

{(25, 18, 5, 7), (25, 18, 5, 7, 20), (16, 9, 14, 6), (0, 2, 26, 28), (25, 18, 5, 7, 23), (24, 11, 29, 22), (27, 17, 10, 19), (8, 1, 3, 13), (4, 12, 15, 21)}


In [10]:
elapsed_time = time.time() - start
print ("elapsed_time:{0}".format(elapsed_time) + "[sec]")

elapsed_time:0.07642006874084473[sec]
